In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df=pd.read_csv('anime.csv')

In [3]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [5]:
df.dropna(subset=['genre','type'],inplace=True)

In [6]:
df['rating'].fillna(df['rating'].mean(),inplace=True)

In [7]:
df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12210 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12210 non-null  int64  
 1   name      12210 non-null  object 
 2   genre     12210 non-null  object 
 3   type      12210 non-null  object 
 4   episodes  12210 non-null  object 
 5   rating    12210 non-null  float64
 6   members   12210 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 763.1+ KB


In [9]:
df.describe()

,anime_id,rating,members
count,12210.000000,12210.000000,1.221000e+04
mean,13936.486486,6.478264,1.817871e+04
std,11398.045316,1.015732,5.498978e+04
min,1.000000,1.670000,5.000000e+00
25%,3460.250000,5.900000,2.290000e+02
50%,10168.500000,6.550000,1.571000e+03
75%,24442.500000,7.170000,9.530000e+03
max,34527.000000,10.000000,1.013917e+06


In [10]:
df_copy = df.copy()

In [11]:
df_copy['genre'] = df_copy['genre'].str.replace(', ', ',')

In [12]:
genres_stacked = df_copy['genre'].str.split(',').apply(pd.Series).stack()

In [13]:
genres_stacked


0      0           Drama
       1         Romance
       2          School
       3    Supernatural
1      0          Action
                ...     
12289  0          Hentai
12290  0          Hentai
12291  0          Hentai
12292  0          Hentai
12293  0          Hentai
Length: 36222, dtype: object

In [14]:
genres_stacked = genres_stacked.astype(str)

In [15]:
genre_matrix_dummies = pd.get_dummies(genres_stacked, prefix='genre')

In [16]:
genre_matrix_dummies

genre_Action  genre_Adventure  genre_Cars  genre_Comedy  \
0     0         False            False       False         False   
      1         False            False       False         False   
      2         False            False       False         False   
      3         False            False       False         False   
1     0          True            False       False         False   
...               ...              ...         ...           ...   
12289 0         False            False       False         False   
12290 0         False            False       False         False   
12291 0         False            False       False         False   
12292 0         False            False       False         False   
12293 0         False            False       False         False   

         genre_Dementia  genre_Demons  genre_Drama  genre_Ecchi  \
0     0           False         False         True        False   
      1           False         False        False        False   
      2           False         False        False        False   
      3           False         False        False        False   
1     0           False         False        False        False   
...                 ...           ...          ...          ...   
12289 0           False         False        False        False   
12290 0           False         False        False        False   
12291 0           False         False        False        False   
12292 0           False         False        False        False   
12293 0           False         False        False        False   

         genre_Fantasy  genre_Game  ...  genre_Shounen Ai  \
0     0          False       False  ...             False   
      1          False       False  ...             False   
      2          False       False  ...             False   
      3          False       False  ...             False   
1     0          False       False  ...             False   
...                ...         ...  ...               ...   
12289 0          False       False  ...             False   
12290 0          False       False  ...             False   
12291 0          False       False  ...             False   
12292 0          False       False  ...             False   
12293 0          False       False  ...             False   

         genre_Slice of Life  genre_Space  genre_Sports  genre_Super Power  \
0     0                False        False         False              False   
      1                False        False         False              False   
      2                False        False         False              False   
      3                False        False         False              False   
1     0                False        False         False              False   
...                      ...          ...           ...                ...   
12289 0                False        False         False              False   
12290 0                False        False         False              False   
12291 0                False        False         False              False   
12292 0                False        False         False              False   
12293 0                False        False         False              False   

         genre_Supernatural  genre_Thriller  genre_Vampire  genre_Yaoi  \
0     0               False           False          False       False   
      1               False           False          False       False   
      2               False           False          False       False   
      3                True           False          False       False   
1     0               False           False          False       False   
...                     ...             ...            ...         ...   
12289 0               False           False          False       False   
12290 0               False           False          False       False   
12291 0               False           False        

In [17]:
genre_matrix = genre_matrix_dummies.groupby(level=0).sum()

In [18]:
genre_matrix

,genre_Action,genre_Adventure,genre_Cars,genre_Comedy,genre_Dementia,genre_Demons,genre_Drama,genre_Ecchi,genre_Fantasy,genre_Game,...,genre_Shounen Ai,genre_Slice of Life,genre_Space,genre_Sports,genre_Super Power,genre_Supernatural,genre_Thriller,genre_Vampire,genre_Yaoi,genre_Yuri
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_copy = df_copy.join(genre_matrix)

In [20]:
df_copy

,anime_id,name,genre,type,episodes,rating,members,genre_Action,genre_Adventure,genre_Cars,...,genre_Shounen Ai,genre_Slice of Life,genre_Space,genre_Sports,genre_Super Power,genre_Supernatural,genre_Thriller,genre_Vampire,genre_Yaoi,genre_Yuri
0,32281,Kimi no Na wa.,"Drama,Romance,School,Supernatural",Movie,1,9.37,200630,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,"Action,Adventure,Drama,Fantasy,Magic,Military,...",TV,64,9.26,793665,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,28977,Gintama°,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,51,9.25,114262,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9253,Steins;Gate,"Sci-Fi,Thriller",TV,24,9.17,673572,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,9969,Gintama&#039;,"Action,Comedy,Historical,Parody,Samurai,Sci-Fi...",TV,51,9.16,151266,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,5543,Under World,Hentai,OVA,1,4.28,183,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_copy.drop(columns=['genre'],inplace=True)

In [22]:
df_copy

,anime_id,name,type,episodes,rating,members,genre_Action,genre_Adventure,genre_Cars,genre_Comedy,...,genre_Shounen Ai,genre_Slice of Life,genre_Space,genre_Sports,genre_Super Power,genre_Supernatural,genre_Thriller,genre_Vampire,genre_Yaoi,genre_Yuri
0,32281,Kimi no Na wa.,Movie,1,9.37,200630,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,28977,Gintama°,TV,51,9.25,114262,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,9253,Steins;Gate,TV,24,9.17,673572,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,9969,Gintama&#039;,TV,51,9.16,151266,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,OVA,1,4.15,211,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,5543,Under World,OVA,1,4.28,183,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,5621,Violence Gekiga David no Hoshi,OVA,4,4.88,219,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,OVA,1,4.98,175,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
col = [col for col in df_copy.columns if col.startswith('genre_')]

In [24]:
df_features= df_copy[col].values

In [25]:
df_features

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [26]:
cosine_sim = cosine_similarity(df_features)

In [27]:
indices = pd.Series(df_copy.index, index=df_copy['name'])

In [28]:
indices

name
Kimi no Na wa.                                            0
Fullmetal Alchemist: Brotherhood                          1
Gintama°                                                  2
Steins;Gate                                               3
Gintama&#039;                                             4
                                                      ...  
Toushindai My Lover: Minami tai Mecha-Minami          12289
Under World                                           12290
Violence Gekiga David no Hoshi                        12291
Violence Gekiga Shin David no Hoshi: Inma Densetsu    12292
Yasuji no Pornorama: Yacchimae!!                      12293
Length: 12210, dtype: int64

In [61]:
def get_recommendations(anime_title, N=10, similarity_threshold=0.75):
    if anime_title not in indices:
        return f"Anime '{anime_title}' not found in the dataset."
    
    idx = indices[anime_title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = [s for s in sim_scores if s[1] >= similarity_threshold]
    sim_scores = sorted(sim_scores,key=lambda x: x[1], reverse=True)
    
    anime_indices = [i[0] for i in sim_scores[1:N+1]]
    similarity_scores = [i[1] for i in sim_scores[1:N+1]]
    
    recommended_anime = df.iloc[anime_indices].copy()
    recommended_anime['similarity_score'] = similarity_scores
    
    recommended_anime = recommended_anime[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members', 'similarity_score']]
    recommended_anime.sort_values(by='similarity_score', ascending=False, inplace=True)
    
    
    return recommended_anime


In [73]:
target_anime = 'Shingeki no Kyojin'
recommendations = get_recommendations(target_anime, N=10,similarity_threshold=0.5)

print(f"\nRecommendations for '{target_anime}':")
recommendations


Recommendations for 'Shingeki no Kyojin':


,anime_id,name,genre,type,episodes,rating,members,similarity_score
717,18397,Shingeki no Kyojin OVA,"Action, Drama, Fantasy, Shounen, Super Power",OVA,3,7.880000,121063,1.000000
961,23777,Shingeki no Kyojin Movie 2: Jiyuu no Tsubasa,"Action, Drama, Fantasy, Shounen, Super Power",Movie,1,7.750000,26958,1.000000
1069,23775,Shingeki no Kyojin Movie 1: Guren no Yumiya,"Action, Drama, Fantasy, Shounen, Super Power",Movie,1,7.700000,31990,1.000000
3533,19285,Shingeki no Kyojin: Ano Hi Kara,"Action, Drama, Fantasy, Shounen, Super Power",Special,1,6.960000,49593,1.000000
10899,25777,Shingeki no Kyojin Season 2,"Action, Drama, Fantasy, Shounen, Super Power",TV,Unknown,6.478264,170054,1.000000
821,3515,Saint Seiya: Meiou Hades Elysion-hen,"Action, Fantasy, Shounen, Super Power",OVA,6,7.820000,19044,0.894427
74,21,One Piece,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",TV,Unknown,8.580000,504862,0.845154
231,19123,One Piece: Episode of Merry - Mou Hitori no Na...,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",Special,1,8.290000,17247,0.845154
241,15323,One Piece: Episode of Nami - Koukaishi no Nami...,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",Special,1,8.270000,24707,0.845154
896,31289,One Piece: Episode of Sabo - 3 Kyoudai no Kizu...,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",Special,1,7.780000,11922,0.845154


In [74]:
target_anime = 'Shingeki no Kyojin'
recommendations = get_recommendations(target_anime, N=10,similarity_threshold=0.85)

print(f"\nRecommendations for '{target_anime}':")
recommendations


Recommendations for 'Shingeki no Kyojin':


,anime_id,name,genre,type,episodes,rating,members,similarity_score
717,18397,Shingeki no Kyojin OVA,"Action, Drama, Fantasy, Shounen, Super Power",OVA,3,7.880000,121063,1.000000
961,23777,Shingeki no Kyojin Movie 2: Jiyuu no Tsubasa,"Action, Drama, Fantasy, Shounen, Super Power",Movie,1,7.750000,26958,1.000000
1069,23775,Shingeki no Kyojin Movie 1: Guren no Yumiya,"Action, Drama, Fantasy, Shounen, Super Power",Movie,1,7.700000,31990,1.000000
3533,19285,Shingeki no Kyojin: Ano Hi Kara,"Action, Drama, Fantasy, Shounen, Super Power",Special,1,6.960000,49593,1.000000
10899,25777,Shingeki no Kyojin Season 2,"Action, Drama, Fantasy, Shounen, Super Power",TV,Unknown,6.478264,170054,1.000000
821,3515,Saint Seiya: Meiou Hades Elysion-hen,"Action, Fantasy, Shounen, Super Power",OVA,6,7.820000,19044,0.894427


In [75]:
target_anime = 'Shingeki no Kyojin'
recommendations = get_recommendations(target_anime, N=10,similarity_threshold=0.9)

print(f"\nRecommendations for '{target_anime}':")
recommendations


Recommendations for 'Shingeki no Kyojin':


,anime_id,name,genre,type,episodes,rating,members,similarity_score
717,18397,Shingeki no Kyojin OVA,"Action, Drama, Fantasy, Shounen, Super Power",OVA,3,7.880000,121063,1.0
961,23777,Shingeki no Kyojin Movie 2: Jiyuu no Tsubasa,"Action, Drama, Fantasy, Shounen, Super Power",Movie,1,7.750000,26958,1.0
1069,23775,Shingeki no Kyojin Movie 1: Guren no Yumiya,"Action, Drama, Fantasy, Shounen, Super Power",Movie,1,7.700000,31990,1.0
3533,19285,Shingeki no Kyojin: Ano Hi Kara,"Action, Drama, Fantasy, Shounen, Super Power",Special,1,6.960000,49593,1.0
10899,25777,Shingeki no Kyojin Season 2,"Action, Drama, Fantasy, Shounen, Super Power",TV,Unknown,6.478264,170054,1.0


## 1. Can you explain the difference between user-based and item-based collaborative filtering?

User-based collaborative filtering finds users with similar preferences and recommends items those users liked.
Item-based collaborative filtering finds items similar to those a user has already liked and recommends them.
User-based focuses on user similarity, while item-based focuses on item similarity and is generally more scalable.

## 2. What is collaborative filtering, and how does it work?

Collaborative filtering is a recommendation technique that predicts a user’s interests based on the preferences of many users.
It works by finding similarities between users or items using past interactions and recommending items with the highest predicted ratings.